In [ ]:
#@title **Mount google drive**

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title **Download video via google drive**

from subprocess import Popen, PIPE, getoutput 
Popen(["pip","install","pytube"]).communicate()
import os
os.system("pip install pytube")
!apt-get install xattr > /dev/null
from pytube import YouTube
from google.colab import files
import os

DRIVE_FOLDER = "MyDrive/Downloads" #@param {type:"string"}
folder = "/content/drive/" + DRIVE_FOLDER + "/"
os.chdir(folder)

#@markdown ---
YT_URL = "https://www.youtube.com/watch?v=RLuCBph9FkQ" #@param {type:"string"}
#@markdown ---
File_Type = "Video" #@param ["Audio", "Video"]
if(File_Type=="Audio"): 
  itag = 140 
else: 
  itag = 18

filename = ""

if(YT_URL.startswith("https://")):
  yt = YouTube(YT_URL)
  yt.streams.get_by_itag(itag).download()
  filename= yt.streams.get_by_itag(itag).default_filename
  os.rename(filename, filename.replace(" ", "-"))
  filename = filename.replace(" ", "-")

  if(File_Type=='Audio'):
    os.rename(filename, filename[0:-3]+"mp3") 
    filename=filename[0:-3]+"mp3"
  
  while(1):
    fid = getoutput("xattr -p"+ " 'user.drive.id' "+"'/content/drive/MyDrive/Downloads/"+filename+"'" )
    if (fid[0:5]=="local"):
        continue
    else:
        print("\n"+filename+":\t"+"https://docs.google.com/uc?export/download&id="+fid)
        break
else:
  print("Enter valid URL")

In [ ]:
#@title **Trim the video based on start and end time**
#@markdown check this if you want to use the previous downloaded video  
use_previous_video = True  # @param {type: "boolean"}
#@markdown ---
#@markdown fill this `input_video` if you want to use another video   
#@markdown    
#@markdown write full path, don't forget the file extension  
input_video = "" #@param {type:"string"} 
#@markdown ---
start_time = "00:01:08" #@param {type:"string"}
end_time = "00:01:23" #@param {type:"string"}
#@markdown ---
trimmed_video_name = filename + "_trimmed.mp4"
trimmed_video_location = os.path.abspath(trimmed_video_name)  

import subprocess

if (use_previous_video):
  input_video_location = os.path.abspath(filename)
else:
  input_video_location = input_video

# Run the ffmpeg command using subprocess
command = f'ffmpeg -i {input_video_location} -ss {start_time} -to {end_time} -c:v libx264 -c:a aac {trimmed_video_name} -y -hide_banner -loglevel error'
result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Check if the command was successful
if result.returncode == 0:
  print("\n"+"Output trimmed video is saved at: "+ trimmed_video_location)
  while(1):
    fid = getoutput("xattr -p"+ " 'user.drive.id' "+"'/content/drive/MyDrive/Downloads/"+trimmed_video_name+"'" )
    if (fid[0:5]=="local"):
      continue
    else:
      print("\n"+trimmed_video_name+":\t"+"https://docs.google.com/uc?export/download&id="+fid)
      break
else:
  print("Error occurred while trimming the video.")
  print("Error message: " + result.stderr.decode('utf-8'))

In [ ]:
#@title **Install packages and libs**

os.chdir("/content/")

!git clone https://github.com/s0md3v/roop --quiet
%cd roop

# !git clone https://github.com/WellAdjustedChimp/roop_unlimited --quiet
# %cd roop_unlimited

!pip install -r requirements.txt --quiet

#for gpu
!pip uninstall onnxruntime --quiet
!pip install onnxruntime-gpu --quiet

#for cpu
# !pip uninstall onnxruntime-gpu --quiet
# !pip install onnxruntime --quiet

!wget https://huggingface.co/MonsterMMORPG/SECourses/resolve/main/inswapper_128.onnx --quiet

!apt-get install ffmpeg --quiet
!apt-get update --quiet
!apt-get upgrade --quiet

In [ ]:
#@title **Prepare the file (important!)**
#@markdown `image_input_location` is full location path of input image
#@markdown ---

import os
os.chdir("/content/roop")
# os.chdir("/content/roop_unlimited")

current_directory = "/content/roop"
# current_directory = "/content/roop_unlimited"
image_input_location = "/content/drive/MyDrive/Downloads/BLB4.jpg" #@param {type:"string"}
image_name = os.path.basename(image_input_location)
image_destination = os.path.join(current_directory, image_name)

trimmed_video_destination = os.path.join(current_directory, trimmed_video_name)
trimmed_video_name_without_extension, _ = os.path.splitext(trimmed_video_name)

# if file not available, copy file from their directory to current folder ("/content/roop")
if not os.path.exists(image_destination):
    !cp {image_input_location} {image_destination}
if not os.path.exists(trimmed_video_destination):
    !cp {trimmed_video_location} {trimmed_video_destination}    

# Apparently `run.py` does not work if the file name is too long,
# so below is my way to rename file, and then replace the file name with the acronym
# Replace '-' with spaces, and split the string into words
list_words = trimmed_video_name.replace('-', ' ').split()
# Create the acronym
acronym = "".join([word[0].upper() for word in list_words])
# Limit the acronym to 6 characters
process_shortname = acronym[:6] + ".mp4"
os.rename(trimmed_video_name, process_shortname)

process_shortname_without_extension, _ = os.path.splitext(process_shortname)
deepfaked_video = process_shortname_without_extension + "_df.mp4"
deepfaked_video_location = os.path.abspath(deepfaked_video)

In [ ]:
#@title **Run deepfake command option 1: via subprocess**

os.chdir("/content/roop")
# os.chdir("/content/roop_unlimited")

import subprocess

# Run the ffmpeg command using subprocess
command = f'python run.py -f {image_name} -t {process_shortname} -o {deepfaked_video} --keep-fps --gpu --keep-frames'
result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Check if the command was successful
if result.returncode == 0:
  print("\n"+"Output deepfake video is saved at: "+ deepfaked_video_location)
else:
  print("Error occurred while processing the video.")
  print("Error message: " + result.stderr.decode('utf-8'))

In [ ]:
#@title **Run deepfake command option 2: directly**

os.chdir("/content/roop")
# os.chdir("/content/roop_unlimited")

# using CPU
# !python run.py -f {image_name} -t {process_shortname} -o {deepfaked_video} --keep-fps --keep-frames

# using GPU
!python run.py -f {image_name} -t {process_shortname} -o {deepfaked_video} --keep-fps --gpu --keep-frames